In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.devices.experimental import TestDevicePythonSim, PlainNumpySimulator

In [2]:
qml.enable_return()

In [3]:
dev = TestDevicePythonSim()

In [4]:
dev_old = qml.device('default.qubit', wires=2)

In [5]:
def tmp_expand_fn(cirq, max_expansion):
    return dev.preprocess(cirq)

def batch_tform(circ):
    return [circ], lambda x: x

dev.batch_execute = dev.execute
dev.batch_transform = batch_tform #dev.preprocess
dev.expand_fn = tmp_expand_fn
dev.shots = None
dev._shot_vector = []

In [6]:
a = qml.numpy.array([1.2, 3.4, 4.5])
ops =[qml.RX(a[0], wires=0),
qml.CNOT(wires=(0,1)),
qml.RY(a[1], wires=1),
qml.RZ(a[2], wires=1)]
m = [qml.expval(qml.PauliX(1))]
     
qs = qml.tape.QuantumScript(ops, m)

In [7]:
grad_scripts, fn = qml.gradients.param_shift(qs)
grad_scripts, fn

([<QuantumScript: wires=[0, 1], params=3>,
  <QuantumScript: wires=[0, 1], params=3>,
  <QuantumScript: wires=[0, 1], params=3>,
  <QuantumScript: wires=[0, 1], params=3>,
  <QuantumScript: wires=[0, 1], params=3>,
  <QuantumScript: wires=[0, 1], params=3>],
 <function pennylane.gradients.parameter_shift._param_shift_new.<locals>.proc_with_validation(results)>)

In [17]:
qml.execute([qs], dev_old, gradient_fn=qml.gradients.param_shift)

ValueError: not enough values to unpack (expected 2, got 1)

In [13]:
qml.execute([qs], dev, gradient_fn=qml.gradients.param_shift)

[RX(1.2, wires=[0]), CNOT(wires=[0, 1]), RY(3.4, wires=[1]), RZ(4.5, wires=[1]), expval(PauliX(wires=[1]))]


ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
@qml.qnode(dev_old, diff_method=qml.gradients.param_shift)
def cold(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

In [19]:
@qml.qnode(dev, diff_method=qml.gradients.param_shift)
def c(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

In [20]:
a = qml.numpy.array([1.2, 3.4, 4.5])

In [21]:
cold(a)

ValueError: not enough values to unpack (expected 2, got 1)

In [11]:
c(a)

[RX(1.2, wires=[0]), CNOT(wires=[0, 1]), RY(3.4, wires=[1]), RZ(4.5, wires=[1]), expval(PauliX(wires=[1]))]


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
cold(a)

In [ ]:
qml.grad(cold)(a)

In [ ]:
qml.grad(c)(a)